In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import pylab as pl
import matplotlib
import matplotlib.pyplot as plt

dfTest = pd.read_csv('ml_case_test_data.csv')
dfTestHist = pd.read_csv('ml_case_test_hist_data.csv')
dfTestOutput = pd.read_csv('ml_case_test_output_template.csv')


In [3]:
# Explore the Testing Datasets
dfTest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4024 entries, 0 to 4023
Data columns (total 32 columns):
id                          4024 non-null object
activity_new                1670 non-null object
campaign_disc_ele           0 non-null float64
channel_sales               2968 non-null object
cons_12m                    4024 non-null int64
cons_gas_12m                4024 non-null int64
cons_last_month             4024 non-null int64
date_activ                  4024 non-null object
date_end                    4024 non-null object
date_first_activ            877 non-null object
date_modif_prod             3979 non-null object
date_renewal                4020 non-null object
forecast_base_bill_ele      877 non-null float64
forecast_base_bill_year     877 non-null float64
forecast_bill_12m           877 non-null float64
forecast_cons               877 non-null float64
forecast_cons_12m           4024 non-null float64
forecast_cons_year          4024 non-null int64
forecast_discount

In [9]:
dfTest[['date_activ','date_end','date_modif_prod','date_renewal']].head(20)

,date_activ,date_end,date_modif_prod,date_renewal
0,2011-09-06,2016-09-06,2015-08-27,2015-09-07
1,2010-12-13,2016-12-31,2010-12-13,2016-01-01
2,2010-09-04,2016-09-04,2015-08-27,2015-09-05
3,2011-01-03,2017-01-03,2011-01-03,2016-01-04
4,2011-10-07,2016-10-07,2015-08-27,2015-10-09
5,2013-02-19,2016-12-17,2014-12-17,2015-12-21
6,2010-02-05,2016-02-05,2015-12-15,2015-02-07
7,2009-06-04,2016-06-04,2015-05-23,2015-06-14
8,2011-07-11,2016-07-11,2015-06-19,2015-07-12
9,2005-04-15,2016-04-15,2005-04-15,2015-04-16


In [11]:
dfTestOutput.mean()

Unnamed: 0           2012.5
Churn_prediction        NaN
Churn_probability       NaN
dtype: float64

In [12]:
dfTestHist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48236 entries, 0 to 48235
Data columns (total 8 columns):
id              48236 non-null object
price_date      48236 non-null object
price_p1_var    47934 non-null float64
price_p2_var    47934 non-null float64
price_p3_var    47934 non-null float64
price_p1_fix    47934 non-null float64
price_p2_fix    47934 non-null float64
price_p3_fix    47934 non-null float64
dtypes: float64(6), object(2)
memory usage: 2.9+ MB


In [13]:
#Join dfTestHist with dfTest1 to Append pricing history 
dfTest2 = pd.merge(dfTest, dfTestHist, on='id', how='inner')
                                   
# Create a set of dummy variables from the sex variable
df_hasgas = pd.get_dummies(dfTest2['has_gas'])

# Join the dummy variables to the main dataframe
dfTest2 = pd.concat([dfTest2, df_hasgas], axis=1)                                    

# Convert date to numeric 
dfTest2['date_activ'] = pd.to_numeric(dfTest2.date_activ.str.replace('-',''))
dfTest2['date_end'] = pd.to_numeric(dfTest2.date_end.str.replace('-',''))
dfTest2['date_first_activ'] = pd.to_numeric(dfTest2.date_first_activ.str.replace('-',''))
dfTest2['date_modif_prod'] = pd.to_numeric(dfTest2.date_modif_prod.str.replace('-',''))
dfTest2['date_renewal'] = pd.to_numeric(dfTest2.date_renewal.str.replace('-',''))
dfTest2['price_date'] = pd.to_numeric(dfTest2.price_date.str.replace('-',''))

# Create a binary variable to indicate missing values
dfTest2['Flag_Miss'] = np.where(dfTest2['date_first_activ'] > 0, 0, 1)

# Drops numeric columns with large % of missing value
dfTest3 = dfTest2.drop(['campaign_disc_ele','date_first_activ','forecast_base_bill_ele','forecast_base_bill_year','forecast_bill_12m','forecast_cons','t'], axis=1)

In [14]:
dfTest3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48236 entries, 0 to 48235
Data columns (total 35 columns):
id                          48236 non-null object
activity_new                20024 non-null object
channel_sales               35580 non-null object
cons_12m                    48236 non-null int64
cons_gas_12m                48236 non-null int64
cons_last_month             48236 non-null int64
date_activ                  48236 non-null int64
date_end                    48236 non-null int64
date_modif_prod             47697 non-null float64
date_renewal                48188 non-null float64
forecast_cons_12m           48236 non-null float64
forecast_cons_year          48236 non-null int64
forecast_discount_energy    47948 non-null float64
forecast_meter_rent_12m     48236 non-null float64
forecast_price_energy_p1    47948 non-null float64
forecast_price_energy_p2    47948 non-null float64
forecast_price_pow_p1       47948 non-null float64
has_gas                     48236 non-n

In [15]:
# Create a set of dummy variables from the channel_sales variable
channel_sales = pd.get_dummies(dfTest3['channel_sales'],prefix='chs_', drop_first=True)

In [16]:
channel_sales.mean()

chs__ewpakwlliwisiwduibdlfmalxowmwpci    0.062567
chs__fixdbufsefwooaasfcxdxadsiekoceaa    0.000498
chs__foosdfpfkusacimwkcsosbicdxkicaua    0.445145
chs__lmkebamcaaclubfxadlmueccxoimlema    0.131271
chs__sddiedcslfslkckwlfkdpoeeailfpeds    0.000249
chs__usilxuppasemubllopkaafesmlibmsdf    0.097396
dtype: float64

In [17]:
# Join the dummy variables to the main dataframe
dfTest3 = pd.concat([dfTest3, channel_sales], axis=1) 

In [18]:
# Create a set of dummy variables from the channel_sales variable
origin_up = pd.get_dummies(dfTest3['origin_up'],prefix='orig_', drop_first=True)

# Join the dummy variables to the main dataframe
dfTest3 = pd.concat([dfTest3, origin_up], axis=1) 
dfTest3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48236 entries, 0 to 48235
Data columns (total 44 columns):
id                                        48236 non-null object
activity_new                              20024 non-null object
channel_sales                             35580 non-null object
cons_12m                                  48236 non-null int64
cons_gas_12m                              48236 non-null int64
cons_last_month                           48236 non-null int64
date_activ                                48236 non-null int64
date_end                                  48236 non-null int64
date_modif_prod                           47697 non-null float64
date_renewal                              48188 non-null float64
forecast_cons_12m                         48236 non-null float64
forecast_cons_year                        48236 non-null int64
forecast_discount_energy                  47948 non-null float64
forecast_meter_rent_12m                   48236 non-null floa

In [23]:
dfTest4 = dfTest3.drop(['activity_new','has_gas','origin_up','channel_sales'], axis=1)

In [24]:
# Rename the self-defined Churn coloumn name to churn_def 
dfTest4.rename(columns={'chs__ewpakwlliwisiwduibdlfmalxowmwpci': 'chs_ew'}, inplace=True)
dfTest4.rename(columns={'chs__fixdbufsefwooaasfcxdxadsiekoceaa': 'chs_fi'}, inplace=True)
dfTest4.rename(columns={'chs__foosdfpfkusacimwkcsosbicdxkicaua': 'chs_fo'}, inplace=True)
dfTest4.rename(columns={'chs__lmkebamcaaclubfxadlmueccxoimlema': 'chs_lm'}, inplace=True)
dfTest4.rename(columns={'chs__sddiedcslfslkckwlfkdpoeeailfpeds': 'chs_sd'}, inplace=True)
dfTest4.rename(columns={'chs__usilxuppasemubllopkaafesmlibmsdf': 'chs_uf'}, inplace=True)

dfTest4.rename(columns={'orig__kamkkxfxxuwbdslkwifmmcsiusiuosws': 'orig_ka'}, inplace=True)
dfTest4.rename(columns={'orig__ldkssxwpmemidmecebumciepifcamkci': 'orig_ld'}, inplace=True)
dfTest4.rename(columns={'orig__lxidpiddsbxsbosboudacockeimpuepw': 'orig_lx'}, inplace=True)
dfTest4.rename(columns={'orig__usapbepcfoloekilkwsdiboslwaxobdp': 'orig_us'}, inplace=True)

In [25]:
print dfTest4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48236 entries, 0 to 48235
Data columns (total 40 columns):
id                          48236 non-null object
cons_12m                    48236 non-null int64
cons_gas_12m                48236 non-null int64
cons_last_month             48236 non-null int64
date_activ                  48236 non-null int64
date_end                    48236 non-null int64
date_modif_prod             47697 non-null float64
date_renewal                48188 non-null float64
forecast_cons_12m           48236 non-null float64
forecast_cons_year          48236 non-null int64
forecast_discount_energy    47948 non-null float64
forecast_meter_rent_12m     48236 non-null float64
forecast_price_energy_p1    47948 non-null float64
forecast_price_energy_p2    47948 non-null float64
forecast_price_pow_p1       47948 non-null float64
imp_cons                    48236 non-null float64
margin_gross_pow_ele        48212 non-null float64
margin_net_pow_ele          48212 non

In [40]:
cols_keep2 = [
'id',
'Flag_Miss',
'orig_ka',
'margin_gross_pow_ele',
'chs_fo',
'date_activ',
'chs_lm',
'date_modif_prod',
'cons_12m',
'cons_gas_12m',
'f',
'net_margin',
'price_p2_var',
'nb_prod_act',
'date_renewal',
'price_p1_fix',
'forecast_price_energy_p1',
'price_date']

In [41]:
dfTest5 = dfTest4[cols_keep2]
dfTest5 = dfTest5.dropna()
dfTest5.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47049 entries, 0 to 48235
Data columns (total 18 columns):
id                          47049 non-null object
Flag_Miss                   47049 non-null int64
orig_ka                     47049 non-null uint8
margin_gross_pow_ele        47049 non-null float64
chs_fo                      47049 non-null uint8
date_activ                  47049 non-null int64
chs_lm                      47049 non-null uint8
date_modif_prod             47049 non-null float64
cons_12m                    47049 non-null int64
cons_gas_12m                47049 non-null int64
f                           47049 non-null uint8
net_margin                  47049 non-null float64
price_p2_var                47049 non-null float64
nb_prod_act                 47049 non-null int64
date_renewal                47049 non-null float64
price_p1_fix                47049 non-null float64
forecast_price_energy_p1    47049 non-null float64
price_date                  47049 non-null 

In [34]:
X_test = dfTest5[cols_keep2]

In [38]:
dfTrainOutput = pd.read_csv('ml_case_training_output.csv')
dfTrainOutput.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16096 entries, 0 to 16095
Data columns (total 2 columns):
id       16096 non-null object
churn    16096 non-null int64
dtypes: int64(1), object(1)
memory usage: 251.6+ KB


In [42]:
# Inner Joint dfTrainOutput with dfTest to Append Churned Flag 
dfTest6 = pd.merge(dfTest5, dfTrainOutput, on='id', how='inner')
dfTest6.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 19 columns):
id                          0 non-null object
Flag_Miss                   0 non-null int64
orig_ka                     0 non-null uint8
margin_gross_pow_ele        0 non-null float64
chs_fo                      0 non-null uint8
date_activ                  0 non-null int64
chs_lm                      0 non-null uint8
date_modif_prod             0 non-null float64
cons_12m                    0 non-null int64
cons_gas_12m                0 non-null int64
f                           0 non-null uint8
net_margin                  0 non-null float64
price_p2_var                0 non-null float64
nb_prod_act                 0 non-null int64
date_renewal                0 non-null float64
price_p1_fix                0 non-null float64
forecast_price_energy_p1    0 non-null float64
price_date                  0 non-null int64
churn                       0 non-null int64
dtypes: float64(7), int64(7), object(1), u

In [45]:
#import pickle
#filename = 'final_model.sav'
#Load the model from disk
#loaded_model = pickle.load(open(filename, 'rb'))
#result = loaded_model.score(X_test)
#print(result)

In [47]:
dfTestOutput.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4024 entries, 0 to 4023
Data columns (total 4 columns):
Unnamed: 0           4024 non-null int64
id                   4024 non-null object
Churn_prediction     0 non-null float64
Churn_probability    0 non-null float64
dtypes: float64(2), int64(1), object(1)
memory usage: 125.8+ KB
